In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
from tensorflow.keras.regularizers import Regularizer
# Figure out how to import regularizers
###
import tensorflow.keras.utils as ku 
import numpy as np 

In [2]:
tokenizer = Tokenizer()
'''!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt'''
data = open('./shakespear_poetry.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [3]:
corpus[1]

"that thereby beauty's rose might never die,"

In [4]:
label

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [5]:
model = Sequential()
model.add(Embedding(total_words, 50, input_length=max_sequence_len-1))  #(# Your Embedding Layer)
model.add(Bidirectional(LSTM(150, return_sequences=True)))  #(# An LSTM Layer)
model.add(Dropout(0.2))  #(# A dropout layer)
model.add(LSTM(100))  #(# Another LSTM Layer)
model.add(Dense(total_words/2, activation='relu'))  #(# A Dense Layer including regularizers)
model.add(Dense(total_words, activation='softmax'))  #(# A Dense Layer)
# Pick an optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')  #(# Pick a loss function and an optimizer)
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 50)            160550    
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 300)           241200    
_________________________________________________________________
dropout (Dropout)            (None, 10, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 1605)              162105    
_________________________________________________________________
dense_1 (Dense)              (None, 3211)              5156866   
Total params: 5,881,121
Trainable params: 5,881,121
Non-trainable params: 0
______________________________________________

In [13]:
 history = model.fit(predictors, label, epochs=250, verbose=1)

Epoch 1/250
484/484 [==============================] - 25s 45ms/step - loss: 6.8260 - accuracy: 0.0232
Epoch 2/250
484/484 [==============================] - 21s 44ms/step - loss: 6.4518 - accuracy: 0.0307
Epoch 3/250
484/484 [==============================] - 22s 45ms/step - loss: 6.2461 - accuracy: 0.0360
Epoch 4/250
484/484 [==============================] - 26s 53ms/step - loss: 6.0417 - accuracy: 0.0404
Epoch 5/250
484/484 [==============================] - 22s 44ms/step - loss: 5.8012 - accuracy: 0.0509
Epoch 6/250
484/484 [==============================] - 22s 44ms/step - loss: 5.5337 - accuracy: 0.0603
Epoch 7/250
484/484 [==============================] - 21s 44ms/step - loss: 5.2752 - accuracy: 0.0720
Epoch 8/250
484/484 [==============================] - 22s 46ms/step - loss: 5.0082 - accuracy: 0.0805
Epoch 9/250
484/484 [==============================] - 22s 46ms/step - loss: 4.7165 - accuracy: 0.0983
Epoch 10/250
484/484 [==============================] - 21s 44ms/step - l

484/484 [==============================] - 21s 44ms/step - loss: 0.5007 - accuracy: 0.8459
Epoch 158/250
484/484 [==============================] - 21s 43ms/step - loss: 0.4893 - accuracy: 0.8472
Epoch 159/250
484/484 [==============================] - 21s 43ms/step - loss: 0.4778 - accuracy: 0.8487
Epoch 160/250
484/484 [==============================] - 21s 43ms/step - loss: 0.4747 - accuracy: 0.8505
Epoch 161/250
484/484 [==============================] - 21s 43ms/step - loss: 0.4855 - accuracy: 0.8479
Epoch 162/250
484/484 [==============================] - 21s 43ms/step - loss: 0.4779 - accuracy: 0.8505
Epoch 163/250
484/484 [==============================] - 21s 43ms/step - loss: 0.4900 - accuracy: 0.8481
Epoch 164/250
484/484 [==============================] - 21s 43ms/step - loss: 0.4874 - accuracy: 0.8467
Epoch 165/250
484/484 [==============================] - 21s 43ms/step - loss: 0.4897 - accuracy: 0.8472
Epoch 166/250
484/484 [==============================] - 21s 43ms/ste

In [6]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [7]:
def plot_graph(history,string):
    plt.plot(history.history[string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.show()

In [8]:
plot_graph(history,'accuracy')

NameError: name 'history' is not defined

In [9]:
predict_x=model.predict(x_test)
classes_x=np.argmax(predict_x,axis=1)

NameError: name 'x_test' is not defined

In [10]:

seed_text = "on the road  "
next_words = 4
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

AttributeError: 'Sequential' object has no attribute 'predict_classes'

SyntaxError: invalid syntax (<ipython-input-37-faadf72f1a65>, line 1)